In [102]:
import s3fs
import xarray as xr
import numpy as np
import coiled

In [103]:
CITYLATLON = {}
with open('ghsl_500k.csv', 'r') as ifile:
    for line in ifile.readlines():
        items = [i.strip() for i in line.split(',')]
        CITYLATLON['city_{0}'.format(items[0])] = (float(items[2]), float(items[3]), int(items[0]))

In [104]:
PERCENTILE_STARTYEAR = 1980
PERCENTILE_ENDYEAR = 2019

In [105]:
ERA_TRANSFORMS = {'air_temperature_at_2_metres': lambda x: x - 273.5, 'precipitation_amount_1hour_Accumulation': lambda x: x * 1000}

In [106]:
def s3open(path):
    fs = s3fs.S3FileSystem(anon=True, default_fill_cache=False)
    return s3fs.S3Map(path, s3=fs)

In [107]:
MONTHS = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

In [ ]:
%%time
def returnperiod_value_daily(timeseries, rp):
    d = timeseries[timeseries > 0.01]  # Only consider actual positive events
    vals, counts = np.unique(d, return_counts=True)
    freqs = counts / d.size
    cdf_y = np.cumsum(freqs)
    targetfreq = (PERCENTILE_ENDYEAR - PERCENTILE_STARTYEAR + 1) / rp
    return np.interp(1-targetfreq, vals, cdf_y)

def pr_rp100_onecity(latlon):
    varname='precipitation_amount_1hour_Accumulation'
    FILE_PATTERN = 'era5-pds/zarr/{year}/{month}/data/{varname}.zarr/'
    files_mapper = [s3open(FILE_PATTERN.format(year=year, month=month, varname=varname)) for year in range(PERCENTILE_STARTYEAR,PERCENTILE_ENDYEAR+1) for month in MONTHS]
    pr_ds = xr.open_mfdataset(files_mapper, engine='zarr', 
           concat_dim=['time0', 'time1'][int(varname=='precipitation_amount_1hour_Accumulation')], combine='nested', 
           coords='minimal', compat='override', parallel=True)
    pr_dailysum = ERA_TRANSFORMS[varname](pr_ds.resample(time1='D').sum())
    ts = pr_dailysum[varname].sel(lat=latlon[0], lon=latlon[1], method='nearest')
    return returnperiod_value_daily(ts.to_numpy(), 100)

cluster = coiled.Cluster(n_workers=25)
client = cluster.get_client()
futs = client.map(pr_rp100_onecity, [(CITYLATLON[c][0], CITYLATLON[c][1]) for c in CITYLATLON])
results = client.gather(futs)
with open('precip_rp100.txt', 'w') as ofile:
    for idx, res in enumerate(results):
        print('{0}:{1}'.format(idx, res), end=' ')
        ofile.write('{0}\t{1}\n'.format(idx, res))


Output()

╭────────────────────────────────────────── Package Info ──────────────────────────────────────────╮
│                             ╷                                                                    │
│   Package                   │ Note                                                               │
│ ╶───────────────────────────┼──────────────────────────────────────────────────────────────────╴ │
│   coiled_local_halfdeg      │ Source wheel built from C:\Users\theodore.wong\halfdeg             │
│                             ╵                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()